In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df1 = df.drop(columns = {'Cabin' ,'Name'})
df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(df1[['Age']])
df1['Age'] = imputer.transform(df1[['Age']])

In [ ]:
df1.info()

In [ ]:
df1.dropna(axis=0,inplace=True)
df1.info()

In [ ]:
#'Sex' , 'Ticket' , 'Embarked'
df1['Ticket'].nunique()       #Ticket need to be dropped as their are too many unique values

In [ ]:
 df1['Sex'].nunique()  

In [ ]:
df1['Embarked'].nunique()

In [ ]:
#OneHot Encode Sex and Embarked columns
df2 = pd.get_dummies(df1[['Sex','Embarked']])
df2.head()

In [ ]:
df1 = df1.join(df2)
df1.head()

In [ ]:
df1.drop(columns={'Sex' , 'Embarked' , 'Ticket'}, axis=1 , inplace=True)
df1.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8), dpi= 80)
sns.heatmap(df1.corr(), cmap='RdYlGn', center=0)

# Decorations
plt.title('Correlogram of mtcars', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
y = df1['Survived']
X = df1.drop(columns={'Survived'}, axis=1)
X.head()

**Manipulating Test Data**

In [ ]:
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
df_test.info()

In [ ]:
df_test.drop(columns={'Cabin','Name','Ticket'},axis=1,inplace=True)
df_test.info()

In [ ]:
df_test2 = pd.get_dummies(df_test[['Sex','Embarked']])
df_test2.head()

df_test = df_test.join(df_test2)
df_test.head()

In [ ]:
df_test.drop(columns={'Sex','Embarked'},axis=1,inplace=True)
df_test['Age'] = imputer.transform(df_test[['Age']])
df_test['Fare'] = imputer.transform(df_test[['Fare']])


df_test.info()

In [ ]:
#Submission Dataframe
sub = pd.DataFrame(columns = ['PassengerId' , 'Survived'])
sub['PassengerId'] = df_test['PassengerId'].astype(int)

In [ ]:
X.drop(columns = ['PassengerId'],axis=1  , inplace= True)

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(learning_rate=0.1, n_estimators=2000, max_depth=15,
                        min_child_weight=0, gamma=0, subsample=0.45, colsample_bytree=1,
                        objective='binary:logistic', nthread=4, scale_pos_weight=1, 
                     reg_alpha=5, reg_lambda=2, booster='gbtree',
            n_jobs=-1, max_delta_step=0, colsample_bylevel=1, colsample_bynode=1)

model = xgb.fit(X, y)
print('train accuracy',xgb.score(X, y))

In [ ]:
df_test.drop(columns=['PassengerId'] , axis=1 , inplace=True)
y_pred = model.predict(df_test)

In [ ]:
sub['Survived'] = y_pred.astype(int)

In [ ]:
sub.to_csv('sub.csv', index=False) 

In [ ]:
col_sorted_by_importance=xgb.feature_importances_.argsort()
feat_imp=pd.DataFrame({
    'cols':X.columns[col_sorted_by_importance],
    'imps':xgb.feature_importances_[col_sorted_by_importance]
})

#!pip install plotly-express
import plotly_express as px
px.bar(feat_imp, x='cols', y='imps')